<h1 align="center">Analisi della qualità del vino Vinho Verde</h1>
<h3 align="center">Stefano Brilli - Marzo 2019</h3>
<hr><hr>

<h2 align="center">Introduzione</h2>

<img src="img/wine.jpg" width="600">

In questo report saranno analizzati dati riguardanti il vino portoghese <b>Vinho Verde</b> nella sua variante rossa, allo scopo di predire se, in base ai valori osservati, si stia osservando un vino di bassa o alta qualità. Il dataset utilizzato è reperibile all'indirizzo http://archive.ics.uci.edu/ml/datasets/Wine+Quality.
Al fine di costruire un classificatore binario, il dataset è stato modificato per rendere la variabile di output <b>(quality)</b> una quantità binaria. In particolare, le osservazioni aventi un output minore od uguale a 5 sono stati etichettati con il <b>valore 0 (bassa qualità)</b> mentre quelli aventi un valore qualitativo di 6 o superiore sono stati etichettati con il <b>valore 1 (alta qualità)</b>.<br>
È chiaro come una mappatura di questo tipo ponga sia i vini con punteggio 6 che quelli con punteggio 10 al pari, nella categoria dei vini di alta qualità. D'altra parte, essa potrebbe permettere ai produttori di avere un iniziale responso della qualità del proprio vino, scartando in prima battuta quelli classificati come di bassa qualità ed andando successivamente ad analizzare nel dettaglio i restanti vini al fine di determinare una scala delle qualità.<br>
I valori di input sono stati calcolati per mezzo di test fisico-chimici, mentre l'output è stato determinato tramite analisi sensoristiche.
Gli attributi del dataset sono i seguenti:

<ul>
  <li>acidità fissa</li>
  <li>acidità volatile</li>
  <li>acido citrico</li>
  <li>zucchero residuo</li>
  <li>cloruri</li>
  <li>anidride solforosa libera</li>
  <li>anidride solforosa totale</li>
  <li>densità</li>
  <li>pH</li>
  <li>solfati</li>
  <li>alcool</li>
</ul>

<hr>
<h3 align="center">Strumenti utilizzati per l'analisi</h3>
L'intera analisi è stata condotta utilizzando il linguaggio <b>Python</b> e l'ambiente <b>Jupyter</b>.<br>
Le librerie utilizzate sono:
<ul>
    <li><b>pandas</b> (<a href="https://pandas.pydata.org/">https://pandas.pydata.org/</a>), libreria software per la manipolazione e l'analisi dei dati</li>
    <li><b>numpy</b> (<a href="http://www.numpy.org/">http://www.numpy.org/</a>), estensione open source del linguaggio di programmazione Python, che aggiunge supporto per vettori e matrici multidimensionali e di grandi dimensioni e con funzioni matematiche di alto livello con cui operare</li>
    <li><b>plotly</b> (<a href="https://plot.ly/feed/">https://plot.ly/feed/</a>), insieme di strumenti per l'analisi e la visualizzazione dei dati</li>
    <li><b>sklearn</b> (<a href="https://scikit-learn.org/stable/">https://scikit-learn.org/stable/</a>), libreria open source di apprendimento automatico per il linguaggio di programmazione Python</li>
</ul>    

<hr><hr>
<h2 align="center">Esplorazione e visualizzazione dei dati</h2>

Il dataset contiene <b>1599 righe</b> e <b>12 colonne</b> (11 di input + 1 di output). Il dataset non è stato alterato nel numero di osservazioni in quanto non risultavano valori mancanti.

In [1]:
# Tesina del corso Data Spaces
# Anno accademico 2018/2019
# Stefano Brilli, matricola s249914

import numpy as np
import pandas as pd


from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA as sklearnPCA

import copy

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly
import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)
from plotly.offline import plot, iplot
import plotly.plotly as py

from IPython.display import Math
import sys

np.set_printoptions(threshold=sys.maxsize)
# Questa funzione legge i dati dal file elimina le righe che contengono valori non numerici e mappa l'output sui valori 0/1
# Se quality <= 5 --> 1
# Altrimenti quality --> 0
def read_data():
    path = '/home/stefano/Documenti/Politecnico/Magistrale/2 Anno/Data Spaces/tesina/wine_quality/'
    df = pd.read_csv(path + 'winequality-red.csv', sep=';')  
    df = df[df.applymap(np.isreal).any(1)]
    df['quality'] = df['quality'].map(lambda x: 1 if x >= 6 else 0) 
    return df

In [2]:
df = read_data()
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,0
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,0
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,1
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,1
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,0


Osservando soltanto queste prime dieci righe possiamo notare uno sbilanciamento tra i vini di bassa qualità e quelli di alta qualità. Ovviamente il campione non può essere preso come rappresentativo dell'intero dataset. Nel grafico seguente viene quindi mostrata la distribuzione delle due etichette di output.

In [3]:
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,0.534709
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.498950
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,0.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,0.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,1.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,1.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,1.000000


In [4]:
colors = ['#2ca02c', '#d62728']
quality_values = {0: "Bassa", 1: "Alta"}

In [5]:
y = df["quality"].value_counts()

data = [go.Bar(x=[quality_values[x] for x in y.index], y=y.values, marker = dict(color = colors[:len(y.index)]))]
layout = go.Layout(
    title='Distribuzione della qualità',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='Numero di osservazioni',
    ),
    xaxis=dict(
        title='Qualità'
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar3')

Come è possibile osservare i dati sono distrubuiti in maniera quasi bilanciata relativamente al valore di output e questo potrebbe significare che il modello di predizione sarà in grado di discriminare bene tra le due classi nella fase di classificazione di nuovi dati. <br>Nel dettaglio, i dati si distribuiscono come segue:
  
<ul>  
    <li><b>Alta qualità</b> (855 su 1599) = <b>53.47%</b></li>
    <li><b>Bassa qualità</b> (744 su 1599) = <b>46.53%</b></li>
</ul>

<hr>
<h3 align="center">Distribuzione dei valori delle feature</h3>

Utilizzando i box plot è possibile osservare in che modo i valori delle feature sono distribuiti. Essi ci permettono di leggerne valore medio, massimo, minimo e rilevare eventuali outlier, ossia dati che non seguono il trend del gruppo a cui appartengono. Se presenti in numero abbastanza elevato, tali valori possono in alcuni casi rendere più complicata l'analisi, perchè il modello apprenderà pattern che si allontanano dal reale andamento dei dati in analisi.
I valori sono stati normalizzati al fine di apprezzare meglio la visualizzazione, in quanto alcune feature presentavano valori estremamente alti rendendo il grafico schiacciato verso il basso.

In [6]:
# Normalizzazione dei dati (tra 0 e 1)
X = df.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
#x_scaled = StandardScaler().fit_transform(X)
df_norm = pd.DataFrame(x_scaled)
#print(df['quality'].values)
#df_norm.columns = df.columns

# Costruzione delle strutture per rappresentare i box plot

data = []
for i in range(len(df.columns) - 1):
    data.append(go.Box(
        name = df.columns[i],
        y = df_norm[i].tolist()        
    ))
df_norm.columns = df.columns    
fig = go.Figure(data=data)
iplot(fig) 

<hr>
<h3 align="center">Correlazione tra le feature</h3>

Il grafico sottostante riporta la correlazione che intercorre tre la feature. Una correlazione positiva indica che quando la prima feature cresce anche la seconda cresce. Una correlazione negativa indica una tendenza inversa all'aumento: quando la prima aumenta la seconda tende a diminuire.

In [7]:
df_ = copy.copy(df_norm).drop(columns=['quality'])
covariance_matrix = df_.corr()
covariance_matrix = covariance_matrix.values
x_ = []
y_ = []
z_ = []

for i in range(11):
    z_.append(covariance_matrix[i,])
    x_.append(df_norm.columns[i])
    y_.append(df_norm.columns[i])

trace = go.Heatmap(z=z_,
                   x=x_,
                   y=y_)    
figure = go.Figure(data=[trace])
iplot(figure)

<hr><hr>
<h2 align="center">Analisi delle componenti principali (PCA)</h2>

L'analisi delle componenti principali (PCA, dall'inglese <b>Principal Components Analysis</b>) è una tecnica che nasce dalla necessità di migliorare le prestazioni degli algoritmi di analisi dei dati a fronte di un enorme incremento della numerosità di essi a disposizione degli analisti. Lo scopo della PCA è dunque quello di rilevare una possibile correlazione tra le diverse variabili di un dataset tale per cui la maggioranza della variabilità dell'informazione dipenda soltanto da esse. Riproiettando i dati lungo gli assi rilevati e scartando le direzioni povere di variabilità è possibile ridurre la dimensionalità del dataset, migliorando le prestazioni ed ottimizzando le risorse.
Per il calcolo delle componenti principali l'approccio è il seguente:
<ul>  
    <li>i dati vengono standardizzati</li>
    <li>si calcolano gli autovalori e gli autovettori dalla matrice di covarianza o da quella di correlazione, oppure si calcola la decomposizione a valore singolo</li>
    <li>si ordinano gli autovalori in ordine decrescente e si calcolano gli autovettori associati ad essi</li>
    <li>si costruisce la matrice W dei dati riproiettati utilizzando gli autovettori selezionati</li>
    <li>si trasforma il dataset originale tramite la matrice W per ottenere un sottospazio Y delle feature di dimensione pari al numero di autovettori utilizzati</li>
</ul>

Nel contesto di questo report sono state utilizzate le funzioni messe a disposizioni da Sklearn per il calcolo della PCA, le quali utilizzano il metodo della decomposizione a valore singolo (SVD) per trovare le componenti principali.

In [8]:
def doPCA(X_std, components, print_flag):    
    X_data = copy.copy(X_std)     
    X_data['quality'] = X_data['quality'].map(lambda x: 'Alta qualità' if x == 1 else 'Bassa qualità')     
    y = X_data['quality']
    X_data = X_data.drop(columns=['quality'])
    #print(X_data.columns)
    #X_data.drop('quality')
    sklearn_pca = sklearnPCA(n_components = components)
    Y_sklearn = sklearn_pca.fit_transform(X_data) 
    #print(Y_sklearn.columns)
    #print(Y_sklearn.shape)
    data = []
    colors = { 'red': '#2ca02c', 'green': '#d62728'}
    for name, col in zip(('Alta qualità', 'Bassa qualità'), colors.values()):
        trace = dict(
            type='scatter',
            x = Y_sklearn[y==name,0],
            y = Y_sklearn[y==name,1],
            mode = 'markers',
            name = name,
            marker = dict(
                color = col,
                size = 12,
                line = dict(
                    color='rgba(217, 217, 217, 0.14)',
                    width=0.5),
                opacity=0.8)
        )
        data.append(trace)

    layout = dict(
            xaxis=dict(title='PC1', showline=True),
            yaxis=dict(title='PC2', showline=True)
    )
    if(print_flag):
        fig = go.Figure(data=data, layout=layout)
        iplot(fig)
    return sklearn_pca

In [9]:
my_pca = doPCA(df_norm, 2, True)
my_total_pca = doPCA(df_norm, 11, False)

Salta immediatamente all'occhio la sovrapposizione dei dati sul nuovo piano di assi PC1 e PC2 e di come non sia possibile separare in cluster separati i gruppi relativi alle due diverse etichette.
Questo significa che non saremo in grado di determinare la qualità dei vini riproiettando i dati su questi due nuovi assi. Il grafico successivo mostra la quantità di varianza raccolta da ciascuna componente, dove possiamo notare come siano sufficienti le prime cinque componenti per spiegare più del 80% della varianza totale.

In [10]:
#print(my_total_pca.explained_variance_ratio_)
def print_explained_variance(explained_variance):    
    cumulative_variance = []
    cumulative_variance.append(explained_variance[0])
    for i in range(1, 11):
        old = cumulative_variance[i-1]
        new = explained_variance[i]
        cumulative_variance.append(old + new)
    
    data = [go.Bar(
            x=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11'],
            y=[explained_variance[0], explained_variance[1], explained_variance[2], 
               explained_variance[3], explained_variance[4], explained_variance[5],
               explained_variance[6], explained_variance[7], explained_variance[8],
               explained_variance[9], explained_variance[10]],
            name="Varianza spiegata"
    ), go.Bar(
            x=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11'],
            y=cumulative_variance,
            name="Varianza cumulativa"
    )]
    layout = go.Layout(
        title='Varianza spiegata',
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig) 

In [11]:
print_explained_variance(my_total_pca.explained_variance_ratio_)

<hr><hr>
<h2 align="center">Classificazione</h2>

Dopo aver osservato i dati secondo la loro distribuzione e correlazione passiamo adesso al costruire un modello predittivo, al fine di prevedere la qualità del vino in base alle sue caratteristiche.
In questo capitolo saranno utilizzati diversi algoritmi di classificazione e, per ciascuno di essi, sarà fornita una adeguata spiegazione teorica, così che anche il lettore non esperto di questo campo possa capire e cogliere le motivazione che stanno dietro allo studio qui presentato.
Gli algoritmi di classificazione che saranno utilizzati sono:
<ul>
    <li><b>K-Nearest Neighbors (KNN)</b></li>
    <li><b>Support Vector Machine (SVM)</b></li>
    <li><b>Random Forest</b></li>
    <li><b>Regressione Logistica</b></li>
</ul>

<hr>
<h3 align="center">K-Nearest Neighbors (KNN)</h3>

Il KNN è un algoritmo utilizzato per costruire e classificare pattern di oggetti basandosi sulle caratteristiche di essi. Tra gli algoritmi utilizzati nel Machine Learning è quello più semplice da implementare e da comprendere.
Il funzionamento è semplice: i voti dei k vicini del campione determinano il gruppo a cui assegnare l'oggetto in esame. Per evitare situazioni di parità è bene scegliere un valore k che non sia un divisore del numero di gruppi distinti presenti nel dataset. Per k = 1 il campione viene assegnato al gruppo dell'oggetto a lui più vicino.
Per scegliere il valore di k più adeguato è necessario valutare la tipologia di dati che si sta esaminando e la quantità di essi. Infatti, scegliere un valore di k elevato garantisce una migliore resistenza al rumore ma rende meno risconoscibili i margini di separazione.
Nel contesto di questa analisi è stato scelto <b>k = 15</b>.

In [73]:
# Funzione per creare training e testing set
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

# Per valutare l'accuratezza della previsione
from sklearn import metrics

# Etichette
y = df['quality'].values

# Features
X = copy.copy(df_norm).drop(columns=['quality']).values
#X = sklearnPCA(n_components = 2).fit_transform(X)

# Splittaggio del dataset in training set e testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

# Creiamo un classificatore con k = 5
knn = KNeighborsClassifier(n_neighbors=15)

# Alleniamo il modello utilizzando il set di training
knn.fit(X_train, y_train)

# Facciamo una previsione utilizzando il set di testing
y_pred = knn.predict(X_test)

# Accuratezza della previsione, ossia la percentuale di volte in cui il classificatore ha fatto una previsione
# corretta
print("Accuratezza:",metrics.accuracy_score(y_test, y_pred))

Accuratezza: 0.7583333333333333


Un'<b>accuratezza</b> di <b>~76%</b> può essere ritenuta buona, considerando che molte delle feature erano risultate scorrelate tra loro (vedere la matrice di correlazione alcuni paragrafi in alto).
Per costruire il classificatore abbiamo utilizzato il dataset nella sua forma originale, cioè quello provvisto di 11 feature. Per tale motivo non siamo in grado di visualizzare sul piano la distribuzione dei punti e i confini di classificazione che il modello ha predetto.
Se invece fossimo interessati a vedere tali confini dovremmo necessariamente diminuire la dimensionalità del dataset fino ad avere due sole dimensioni.
Per fare ciò utilizziamo ancora la PCA, in modo che il classificatore faccia previsioni basandosi su due sole feature (PC1 e PC2).

In [74]:
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

h = .02
data = []

# Mappa dei colori
cmap_light =[[0, '#FFAAAA'], [0.5, '#AAFFAA'], [1, '#79fb60']]
cmap_bold = [[0, '#FF0000'], [0.5, '#00FF00'], [1, '#0000FF']]
colorsIdx = {0: 'red', 1: 'green'}
legendIdx = {0 : 'Bassa qualità', 1 : 'Alta qualità'}
cols = df['quality'].map(colorsIdx)
leg = df['quality'].map(legendIdx)

y = df['quality'].values
X = copy.copy(df_norm).drop(columns=['quality']).values
X = sklearnPCA(n_components = 2).fit_transform(X)

X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X, y, test_size=0.3)

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_pca, y_train_pca)

x_min, x_max = X[:, 0].min() - 0.10, X[:, 0].max() + 0.10
y_min, y_max = X[:, 1].min() - 0.10, X[:, 1].max() + 0.10
x_ = np.arange(x_min, x_max, h)
y_ = np.arange(y_min, y_max, h)
xx, yy = np.meshgrid(x_, y_)
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])

# Risultato nel grafico dei colori
Z = Z.reshape(xx.shape)
p1 = go.Heatmap(x=x_, y=y_, z=Z,
                showscale=False,
                colorscale=cmap_light)

# Plottiamo anche i dati di training
p2 = go.Scatter(x=X[:, 0], y=X[:, 1], 
                mode='markers',                
                marker=dict(color=cols,
                            colorscale=cmap_bold,
                            line=dict(color='black', width=1)))
data.append(p1)
data.append(p2) 

layout_knn = go.Layout(
        title='KNN con k = 15'
    )

fig_knn = go.Figure(data=data, layout=layout_knn)
iplot(fig_knn) 

# Facciamo una previsione utilizzando il set di testing
y_pred = knn.predict(X_test_pca)
print("Accuratezza sul testing set:", metrics.accuracy_score(y_test_pca, y_pred))

Accuratezza sul testing set: 0.69375


In questo caso l'<b>accuratezza</b> è <b>pari a ~67.7%.</b>
Un risultato del genere non deve sorprendere, perchè come abbiamo visto nel paragrafo dedicato alla PCA le prime componenti raccolgono una buona parte della variabilità del dataset, riuscendo quindi a descriverlo bene anche su poche dimensioni.

<hr>
<h3 align="center">Support Vector Machine (SVM)</h3>

L'SVM è un algoritmo supervisionato per classificare elementi appartenenti a due o più classi. L'obiettivo è quello di trovare l'iperpiano che sia in grado di separare i dati nel modo migliore. Questo significa assegnare, per ciascun punto, un etichetta in base a dove il punto si trovi rispetto all'iperpiano di separazione.
Per trovare tale iperpiano abbiamo bisogno di risolvere un problema di ottimizzazione.


<img src="img/svm_1.png" width="400">

<hr>
<h4 align="center">Il problema di ottimizzazione</h4>

Se i dati sono perfettamente separabili nello spazio allora è dimostrato come sia possibile trovare l'iperpiano di separazione in un numero finito di passi. Ciò è garantito dal <b>teorema di convergenza del Perceptron</b>
(<a href="http://www.cs.columbia.edu/~mcollins/courses/6998-2012/notes/perc.converge.pdf">http://www.cs.columbia.edu/~mcollins/courses/6998-2012/notes/perc.converge.pdf</a>).<br>
A questo punto viene naturale chiedersi quale dei possibili iperpiani dobbiamo scegliere per separare i dati secondo le etichette. L'SVM per definizione ricerca l'iperpiano che massimizza la distanza tra i due gruppi di elementi (si veda l'immagine sopra).<br>
$$ \text{ Formulazione primale: } \min\frac{1}{2}||\omega||^2 \text{ subject to: } y^{(t)}(\omega \cdot x^{(t)} + b) \geq k, \ t=1,...,n $$ <br>
Si tratta di un <b>problema di ottimizzazione quadratico e con vincolo</b>. Esso può essere risolto utilizzando il metodo dei <b>moltiplicatori di Lagrange</b>, ed essendo un problema quadratico si tratta di trovare il minimo globale (unico) di una superficie paraboloidale.
Calcolando le derivate parziali della formulazione primale otteniamo la formulazione alternativa del problema. Essa è definita come $$ \text{ Formulazione alternativa: } \max\text{-}\frac{1}{2}\sum_{ij}{\alpha_i\alpha_j y_i y_j \vec{x_i} \cdot \vec{x_j}} + \sum_i{\alpha_i} \text{ subject to: } \sum_i{\alpha_i y_i} = 0 \text{, } \alpha_i > 0 $$

Formulando il problema in questo modo siamo in grado di calcolare la soluzione elimando la dipendenza dal vettore omega dei pesi e dal bias b. 
L'unica dipendenza rimasta per la risoluzione è quella relativa ai coefficienti alfa.
Questi coefficienti sono per lo più uguali a zero. Gli unici diversi da zero sono quelli relativi agli elementi presenti sui margini, che prendono il nome di <b>support vectors</b>, da cui il nome dell'algoritmo.

Nel caso in cui i dati non siano separabili linearmente, come in questo caso, possiamo pensare di risolvere il problema di ottimizzazione permettendo ad alcuni elementi del dataset di essere dalla parte sbagliata del margine, o addirittura dalla parte sbagliata dell'iperpiano. Questo si formalizza aggiungengo un ulteriore vincolo alle formulazioni precedenti in modo tale da limitare superiormente il limite d'errore a cui il modello può arrivare nella fase di classificazione.
Indichiamo tale valore con <b>C</b>.

<hr>
<h4 align="center">SVM Lineare</h4>

In questa prima applicazione dell'SVM utilizziamo un kernel lineare, ossia cerchiamo quell'iperpiano che massimizza la distanza tra i due cluster di dati. Non siamo in grado, a priori, di determinare quale sia il valore C che massimizza l'accuratezza sul set di testing. Per trovarlo possiamo creare un modello per un insieme di tali valori e scegliere il migliore osservando i risultati ottenuti. <br>
I valori di C con cui sarà testata l'accuratezza sono <b>{0.001, 0.01, 0.1, 1, 10, 1000}</b>.

In [180]:
from sklearn import svm

def svm_linear_analysis(X_train, y_train, X_test, y_test):
    accuracy_array = []
    best_accuracy = 0
    c_best = 0
    for i, cost in enumerate((0.001, 0.01, 0.1, 1, 10, 100, 1000)):    
        # Settiamo il kernel come lineare con C = cost
        clf = svm.SVC(kernel='linear', C=cost)

        # Costruiamo il modello con i dati di training
        clf.fit(X_train, y_train)

        # Testiamo il modello con i dati di testing e stampiamo l'accuratezza
        accuracy = clf.score(X_test, y_test)
        if(accuracy > best_accuracy):
            best_accuracy = accuracy
            c_best = cost
        accuracy_array.append((cost,accuracy))
        #print("Accuratezza con C={}: {}".format(cost, accuracy))
    return round(best_accuracy, 3)*100, c_best, accuracy_array

# Funzione per disegnare l'andamento dell'accuratezza in base al valore di C
def plotSplineFunction(array):
    x = array[:,0]
    y = array[:,1]
    xi = list(i for i in range(0, len(x)))
    
    N = 10

    trace = go.Scatter(
        x = x,
        y = y
    )
    
    layout_svm_linear = dict(
              title="Andamento dell'accuratezza al variare di C",
              yaxis = dict(zeroline = False, title="Accuratezza"),
              xaxis = dict(title="C", type="log", tickmode="linear", tick0=-3, dtick=1)
             )

    data = [trace]

    fig_svm_linear = go.Figure(data=data, layout=layout_svm_linear)
    iplot(fig_svm_linear) 

In [181]:
best_accuracy_linear, c_linear, accuracy_array= svm_linear_analysis(X_train, y_train, X_test, y_test)
print("\n*** Migliore accuratezza trovata con C={}: {}%".format(c_linear, best_accuracy_linear))
plotSplineFunction(np.asarray(accuracy_array))


*** Migliore accuratezza trovata con C=10: 74.8%


<hr>
<h4 align="center">SVM con kernel RBF</h4>

Ciò che possiamo fare per migliorare l'accuratezza del nostro classificatore è quello di passare ad un kernel non lineare, ossia trovare un margine di separazione utilizzando una funzione non lineare.<br>
Uno dei kernel maggiormente utilizzati nell'ambito del SVM è il cosiddetto <b>Radial basis function kernel (RBF Kernel)</b>. La formulazione standard del kernel è $$ K(x,y) = exp(-\frac{\|x - y\|^2}{2\sigma^2}) $$ che possiamo riscrivere come $$ K(x,y) = exp(-\gamma\|x - y\|^2), \quad \gamma = \frac{1}{2\sigma^2} $$

Questa volta dobbiamo fare tuning anche sul parametro gamma oltre che su C, in modo tale che la migliore combinazione delle due fornisca la più alta accuratezza possibile.

In [172]:
import math

def svm_rbf_analysis(X_train, y_train, X_test, y_test):
    accuracy_array = []
    c_array = []
    gamma_array = []
    best_accuracy = 0
    c_best = 0
    gamma_best = 0
    for i, cost in enumerate((0.001, 0.01, 0.1, 1, 10, 100, 1000)):    
        for j, gamma in enumerate((0.002, 0.02, 0.2, 2, 20, 200, 2000)):
            # Settiamo il kernel come rbf con C = cost e Gamma = g
            clf = svm.SVC(kernel='rbf', C=cost, gamma=gamma)

            # Costruiamo il modello con i dati di training
            clf.fit(X_train, y_train)

            # Testiamo il modello con i dati di testing e stampiamo l'accuratezza
            accuracy = clf.score(X_test, y_test)
            c_array.append(cost)
            gamma_array.append(gamma)
            accuracy_array.append(accuracy)
            if(accuracy > best_accuracy):
                best_accuracy = accuracy
                c_best = cost
                gamma_best = gamma
            #print("Accuratezza con C={} e Gamma={}: {}".format(cost, gamma, accuracy))
    return round(best_accuracy, 3)*100, c_best, gamma_best, c_array, gamma_array, accuracy_array

# Funzione per disegnare una heatmap relativa alla variazione di accuratezza al variare di C e Gamma
def plotHeatMapSVM(c_array, gamma_array, accuracy_array):
    trace = go.Heatmap(z=accuracy_array,
                       x=gamma_array,
                       y=c_array)   
    layout_svm_rbf = dict(
          title="Andamento dell'accuratezza al variare di C e Gamma",
          yaxis = dict(zeroline = False, title="C", type="log", tickmode="linear", tick0=-3, dtick=1),
          xaxis = dict(title="Gamma", type="log", tickmode="linear", tick0=math.log10(gamma_array[0]), dtick="log_10(2)")
         )

    figure = go.Figure(data=[trace], layout=layout_svm_rbf)
    iplot(figure)

In [183]:
best_accuracy_rbf, c_rbf, gamma_rbf, c_array_rbf, gamma_array_rbf, accuracy_array_rbf = svm_rbf_analysis(X_train, y_train, X_test, y_test)
print("\n*** Migliore accuratezza trovata con C={} e Gamma={}: {}%".format(c_rbf, gamma_rbf, best_accuracy_rbf))
plotHeatMapSVM(np.asarray(c_array_rbf), np.asarray(gamma_array_rbf), np.asarray(accuracy_array_rbf))


*** Migliore accuratezza trovata con C=100 e Gamma=2: 81.69999999999999%


A fronte di un costo computazionale maggiore si ottiene un'accuratezza decisamente maggiore rispetto a quella ottenuta con un kernel lineare o utilizzando l'algoritmo KNN.

<hr>
<h3 align="center">Regressione logistica</h3>

Nonostante il nome la regressione logistica, anche chiamata <b>regressione logit</b>, è un algoritmo di classificazione.<br>
A differenza di altri algoritmi esso fornisce un risultato sulla base di un valore di probabilità e questo implica che il modello utilizzato deve necessariamente restituire un valore sempre compreso tra 0 e 1.<br>
A tal proposito la regressione logistica utilizza una funzione chiamata <b>Sigmoid function</b>, la cui particolarità è quella di avere codominio compreso tra 0 ed 1.
$$ h_ \theta (x) =  \frac{\mathrm{1} }{\mathrm{1} + e^- \theta^Tx }  $$ 
<img src="img/Logistic_curve.svg" width="600">

<hr>
<h4 align="center">Maximum Likelihood Estimation (MLE) e Gradient Ascent</h4>

Il modello che vogliamo costruire è un iperpiano che separa i dati in due gruppi e che denotiamo con la lettera $\theta$. La likelihood (probabilità) di un dati modello $\theta$ per un certo dataset $\vec{X}, \vec{y}$ è definito come $L(\theta) = P(\vec{y}|\vec{X}; \theta)$.<br>
La MLE ci dice che tra tutti i possibili iperpiani $\theta$ che possiamo trovare, noi cerchiamo quello che <b>massimizza</b> la probabilità per questi dati. Per fare ciò dobbiamo espandere la definizione di likelihood riportata sopra come $$L(\theta) = P(\vec{y}|\vec{X}; \theta) = \prod_i{P(y^{(i)}|x^{i}; \theta)}  $$
dato che la probabilità di un insieme di dati è uguale al prodotto delle probabilità dei singoli elementi.<br>
Quando y, cioè l'etichetta, è uguale a 1 allora $P(y^{(i)}|x^{i}; \theta) = h_ \theta (x)$.<br>
Viceversa quando y = 0 allora abbiamo che $P(y^{(i)}|x^{i}; \theta) = 1 - h_ \theta (x)$

Inserendo quest'ultima equazione in quella della likelihood e passando in notazione logaritmica otteniamo 
$$ log L(\theta) = \sum_i{y^{(i)} \cdot log h_\theta(x^{(i)})+(1-y^{i}) \cdot log(1-h_\theta(x^{(i)}))}$$

L'obiettivo è trovare il valore di $\theta$ (l'iperpiano) per il quale la log-likelihood è massima rispetto a tutti gli altri iperpiani. Essendo tale equazione la somma di un numero arbitrario di parametri non è possibile determinarne il massimo semplicemente calcolandone le derivate parziali. A tal scopo si utilizza un algoritmo di ottimizzazione chiamato <b>Gradient Ascent</b>, che permette di trovare il massimo della funzione per spostandosi sulla superficie da essa definita in maneria graduale e per passaggi successivi.

<hr>
<h4 align="center">Analisi dei dati usando la regressione logistica</h4>

Dopo aver fornito una, seppur superficiale, spiegazione teorica per quanto riguarda la regressione logistica siamo adesso in grado di applicarla al nostro dataset.

In [190]:
from sklearn.linear_model import LogisticRegression

# Applicazione della regressione logistica
def performLogRegression(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial')
    
    # Costruiamo il modello con i dati di training
    clf.fit(X_train, y_train)

    # Testiamo il modello con i dati di testing e stampiamo l'accuratezza
    accuracy = clf.score(X_test, y_test)
    
    return round(accuracy,3)*100
accuracy = performLogRegression(X_train, X_test, y_train, y_test)
print("*** Accuratezza della regressione logistica: {}%".format(accuracy))

*** Accuratezza della regressione logistica: 74.6%
